In [1]:
import pypowsybl as pp

Configure logger to show information

In [2]:
import logging
logging.basicConfig()
logging.getLogger('powsybl').setLevel(logging.INFO)

Create an IEEE 14 network instance

In [3]:
n = pp.network.create_ieee14()

INFO:powsybl:Using platform configuration provider classic
INFO:powsybl:Platform configuration defined by YAML file /home/jamgotchiangeo/.itools/config.yml


Configure load flow parameters to fast restart mode. In this mode, many load flow context data are saved between two run to allow faster restart.

Supported modifications in fast restart are:
  - target voltage of generators.
  - section count of shunt compensators.
  - open/close of switches.
  - target voltage and tap position of transformers (2 and 3 windings).
  - secondary voltage control pilot point target voltage and control units participation.

All others modifications wil result by a full cache cleaning and so on normal execution performances. 

In [4]:
p = pp.loadflow.Parameters(provider_parameters={'networkCacheEnabled': 'true'})

Run first load flow, flat start mode (this is the default behaviours). It converges in 3 iterations. We can see thanks to the log "Network cache created for network 'ieee14cdf'" that cache is first initialized for this network.

In [5]:
pp.loadflow.run_ac(n, p)

INFO:powsybl:loadflow provider used is : OpenLoadFlow
INFO:powsybl:LocalComputationConfig [localDir=/tmp, availableCore=1]
INFO:powsybl:Version: {repositoryName=powsybl-open-loadflow, mavenProjectVersion=1.9.0, gitVersion=3b3973fa5b771f67791a12d741c38a8f22e67f94, gitBranch=UNKNOWN, buildTimestamp=2024-04-04T10:35:32.204Z}
INFO:powsybl:Parameters:
┌───────────────────────────────────────────────────────────┬──────────────────────────────────┐
│ Name                                                      │ Value                            │
├───────────────────────────────────────────────────────────┼──────────────────────────────────┤
│ voltageInitMode                                           │ UNIFORM_VALUES                   │
│ transformerVoltageControlOn                               │ false                            │
│ useReactiveLimits                                         │ true                             │
│ phaseShifterRegulationOn                                  │ false  

[ComponentResult(connected_component_num=0, synchronous_component_num=0, status=CONVERGED, status_text=CONVERGED, iteration_count=3, reference_bus_id='VL1_0', slack_bus_results=[SlackBusResult(id='VL1_0', active_power_mismatch=-0.006730108618313579)], distributed_active_power=0.0)]

Change section count of shunt compensator 'B9-SH' from 1 to 0 (so this is like a shunt disconnection)

In [6]:
n.update_shunt_compensators(id='B9-SH', section_count=0)
n.get_shunt_compensators().filter(items=['section_count'])

,section_count
id,
B9-SH,0


Re-run the load flow. It converges in 3 iterations. We can see that previous created cache for this network has been reused "Network cache reused for network 'ieee14cdf'"

In [7]:
pp.loadflow.run_ac(n, p)

INFO:powsybl:loadflow provider used is : OpenLoadFlow
INFO:powsybl:Version: {repositoryName=powsybl-open-loadflow, mavenProjectVersion=1.9.0, gitVersion=3b3973fa5b771f67791a12d741c38a8f22e67f94, gitBranch=UNKNOWN, buildTimestamp=2024-04-04T10:35:32.204Z}
INFO:powsybl:Parameters:
┌───────────────────────────────────────────────────────────┬──────────────────────────────────┐
│ Name                                                      │ Value                            │
├───────────────────────────────────────────────────────────┼──────────────────────────────────┤
│ voltageInitMode                                           │ UNIFORM_VALUES                   │
│ transformerVoltageControlOn                               │ false                            │
│ useReactiveLimits                                         │ true                             │
│ phaseShifterRegulationOn                                  │ false                            │
│ twtSplitShuntAdmittance                

[ComponentResult(connected_component_num=0, synchronous_component_num=0, status=CONVERGED, status_text=CONVERGED, iteration_count=3, reference_bus_id='VL1_0', slack_bus_results=[SlackBusResult(id='VL1_0', active_power_mismatch=0.15687212390895766)], distributed_active_power=0.0)]

Update generator 'B1-G' target voltage to 144 Kv

In [8]:
n.update_generators(id='B1-G', target_v=144)
n.get_generators().filter(items=['target_v'])

,target_v
id,
B1-G,144.000
B2-G,141.075
B3-G,136.350
B6-G,12.840
B8-G,21.800


Re-run the load flow. It converges in 1 iterations. Again, we can see that previous created cache for this network has been reused thanks to the log "Network cache reused for network 'ieee14cdf'"

In [9]:
pp.loadflow.run_ac(n, p)

INFO:powsybl:loadflow provider used is : OpenLoadFlow
INFO:powsybl:Version: {repositoryName=powsybl-open-loadflow, mavenProjectVersion=1.9.0, gitVersion=3b3973fa5b771f67791a12d741c38a8f22e67f94, gitBranch=UNKNOWN, buildTimestamp=2024-04-04T10:35:32.204Z}
INFO:powsybl:Parameters:
┌───────────────────────────────────────────────────────────┬──────────────────────────────────┐
│ Name                                                      │ Value                            │
├───────────────────────────────────────────────────────────┼──────────────────────────────────┤
│ voltageInitMode                                           │ UNIFORM_VALUES                   │
│ transformerVoltageControlOn                               │ false                            │
│ useReactiveLimits                                         │ true                             │
│ phaseShifterRegulationOn                                  │ false                            │
│ twtSplitShuntAdmittance                

[ComponentResult(connected_component_num=0, synchronous_component_num=0, status=CONVERGED, status_text=CONVERGED, iteration_count=1, reference_bus_id='VL1_0', slack_bus_results=[SlackBusResult(id='VL1_0', active_power_mismatch=-0.02403856689623396)], distributed_active_power=0.0)]

We are now going to demonstrate how to change a switch position in fast restart mode.
First, let's extend the IEEE 14 with an additionnal bus with just a load connected to the initial network with a switch (initially open)

In [10]:
n.create_buses(id='B', voltage_level_id='VL10')
n.create_loads(id='LD', voltage_level_id='VL10', bus_id='B', p0=10, q0=5)
n.create_switches(id='SW', voltage_level_id='VL10', bus1_id='B10', bus2_id='B', open=True)

To be able to operate a switch in fast restart mode, we need to specify the list of switch IDs that might be open or closed in advance

In [11]:
p = pp.loadflow.Parameters(provider_parameters={'networkCacheEnabled': 'true', 'actionableSwitchesIds': 'SW'})

We can now re-run the load flow. We can see that because we have changed the network structure (adding new elements) and also changed the load flow parameters, the cache has been invalidated "Network cache evicted..." and a new one has been created "Network cache created for network 'ieee14cdf'".

In [12]:
pp.loadflow.run_ac(n, p)

INFO:powsybl:loadflow provider used is : OpenLoadFlow
INFO:powsybl:Version: {repositoryName=powsybl-open-loadflow, mavenProjectVersion=1.9.0, gitVersion=3b3973fa5b771f67791a12d741c38a8f22e67f94, gitBranch=UNKNOWN, buildTimestamp=2024-04-04T10:35:32.204Z}
INFO:powsybl:Parameters:
┌───────────────────────────────────────────────────────────┬──────────────────────────────────┐
│ Name                                                      │ Value                            │
├───────────────────────────────────────────────────────────┼──────────────────────────────────┤
│ voltageInitMode                                           │ UNIFORM_VALUES                   │
│ transformerVoltageControlOn                               │ false                            │
│ useReactiveLimits                                         │ true                             │
│ phaseShifterRegulationOn                                  │ false                            │
│ twtSplitShuntAdmittance                

[ComponentResult(connected_component_num=0, synchronous_component_num=0, status=CONVERGED, status_text=CONVERGED, iteration_count=4, reference_bus_id='B1', slack_bus_results=[SlackBusResult(id='B1', active_power_mismatch=0.01692772523167463)], distributed_active_power=0.0)]

Now, we can update close the switch

In [13]:
n.update_switches(id='SW', open=False)

And once again, re-run the load low and see that we restart from the cache "Network cache reused for network 'ieee14cdf'"

In [14]:
pp.loadflow.run_ac(n, p)

INFO:powsybl:loadflow provider used is : OpenLoadFlow
INFO:powsybl:Version: {repositoryName=powsybl-open-loadflow, mavenProjectVersion=1.9.0, gitVersion=3b3973fa5b771f67791a12d741c38a8f22e67f94, gitBranch=UNKNOWN, buildTimestamp=2024-04-04T10:35:32.204Z}
INFO:powsybl:Parameters:
┌───────────────────────────────────────────────────────────┬──────────────────────────────────┐
│ Name                                                      │ Value                            │
├───────────────────────────────────────────────────────────┼──────────────────────────────────┤
│ voltageInitMode                                           │ UNIFORM_VALUES                   │
│ transformerVoltageControlOn                               │ false                            │
│ useReactiveLimits                                         │ true                             │
│ phaseShifterRegulationOn                                  │ false                            │
│ twtSplitShuntAdmittance                

[ComponentResult(connected_component_num=0, synchronous_component_num=0, status=CONVERGED, status_text=CONVERGED, iteration_count=3, reference_bus_id='B1', slack_bus_results=[SlackBusResult(id='B1', active_power_mismatch=11.353082185766317)], distributed_active_power=0.0)]